# Using DESeq2 for eQTLs

In [1]:
suppressMessages(library(edgeR))
suppressMessages(library(DESeq2))
suppressMessages(library('foreach'))
suppressMessages(library('doParallel'))

In [2]:
data_path <- '/Data/lupus/'


In [3]:
snps <- as.data.frame(t(read.table(paste(data_path, 'mateqtl_input/asian_filtered_genos.tsv', sep=''), sep='\t', header=1, row.names=1, check.names = FALSE)))

In [4]:
dim(snps)

[1]   97 7434

In [5]:
ct <- 'T4'

In [6]:
data <- t(read.table(paste(data_path, 'pseudobulk/asian_', ct, '_sum.csv', sep=''), sep='\t', header=1, row.names=1))

In [7]:
cov = as.data.frame(t(read.table(paste(data_path, 'mateqtl_input/asian_mateqtl_cov.txt', sep=''), sep='\t', header=1, row.names=1, check.names=FALSE)))

In [8]:
snps_to_check <- read.table('one1k_T4_eqtl_snps.csv', header=1)[[1]]

In [9]:
# Set up parallel computing

parallel::detectCores()
n.cores <- 40

#create the cluster
my.cluster <- parallel::makeCluster(
  n.cores, 
  type = "FORK"
  )

#check cluster definition (optional)
print(my.cluster)

[1] 48

socket cluster with 40 nodes on host ‘localhost’


In [10]:
#register it to be used by %dopar%
doParallel::registerDoParallel(cl = my.cluster)

#check if it is registered (optional)
foreach::getDoParRegistered()

[1] TRUE

In [11]:
start <- Sys.time()
returned <-  foreach(i=1:length(snps_to_check)) %dopar% {
    snp_var <- snps_to_check[i]
    treat_cov = cov
    treat_cov['snp'] <- snps[[snp_var]]

    dds <- DESeqDataSetFromMatrix(
        countData = round(data),
        colData = treat_cov, 
        design= ~snp)
    dds <- suppressMessages(DESeq(dds))
    res <- results(dds, name="snp")
    write.csv(res, paste(data_path, 'deseq2_output/onek1k_T4_deseq2_',snp_var, '.csv', sep=''))

  }
end <- Sys.time()
print(start-end)

Time difference of -50.30496 mins


In [12]:
start <- Sys.time()
returned <-  foreach(i=1:length(snps_to_check)) %dopar% {
    snp_var <- snps_to_check[i]
    treat_cov = cov
    treat_cov['snp'] <- snps[[snp_var]]

    dds <- DESeqDataSetFromMatrix(
        countData = round(data),
        colData = treat_cov, 
        design= as.matrix(treat_cov))
    dds <- suppressMessages(DESeq(dds))
    res <- results(dds, name="snp")
    write.csv(res, paste(data_path, 'deseq2_output/onek1k_cM_deseq2_allcov_',snp_var, '.csv', sep=''))

  }
end <- Sys.time()
print(start-end)

Time difference of -1.277886 hours


In [ ]:
1.277886